In [9]:
import pandas as pd
import numpy as np
import dgl
import createGraph
from importlib import reload
import torch
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import architecture
import training_loop

In [10]:
# Assign new indices to 2nd graph
def offset_node_batches(node_batches, offset):
    return [[idx + offset for idx in batch] for batch in node_batches]

In [11]:
# Load in graphs directly from binary files
train_g, train_labels_dict = dgl.load_graphs('upsampledTrain.bin')
val_g, val_labels_dict = dgl.load_graphs('nonupsampledTest.bin')

# Assign labels 
train_y = train_labels_dict['gLabel']
val_y = val_labels_dict['gLabel']

# Make a list of cumulative sums of nodes for each graph in training and val set
train_cumsum_nodes = np.cumsum([g.number_of_nodes() for g in train_g])
val_cumsum_nodes = np.cumsum([g.number_of_nodes() for g in val_g])

In [12]:
# Create a list of lists containg indices of nodes for each graph in training and val set
train_node_batches = [list(range(train_cumsum_nodes[i - 1] if i > 0 else 0, train_cumsum_nodes[i])) for i in range(len(train_g))]
val_node_batches = [list(range(val_cumsum_nodes[i - 1] if i > 0 else 0, val_cumsum_nodes[i])) for i in range(len(val_g))]

In [13]:
# Last indexed value from train_cumsum_nodes is total number of nodes in training set
num_train_nodes = train_cumsum_nodes[-1]
# Add number of train nodes to each idx in val set 
offset_val_node_batches = offset_node_batches(val_node_batches, num_train_nodes)
# Combine into single graph
combined_g = dgl.batch(train_g + val_g)
# Get total number of batches/indices 
combined_node_batches = train_node_batches + offset_val_node_batches
# Combine labels for train and val into single tensor
combined_y = torch.cat([train_y, val_y])
# Mask to select training set
train_mask = list(range(len(train_y)))
# Mask to select val set
val_mask = list(range(len(train_y), len(train_y) + len(val_y)))

In [14]:
reload(architecture)
reload(training_loop)

model = architecture.GCN(combined_g.ndata['feat'].shape[1], 30, 30)

training_loop.train(g=combined_g, node_batches=combined_node_batches, model=model, labels=combined_y, train_mask=train_mask, val_mask=val_mask, validate=True, test=False)

C:\Users\james\AppData\Roaming\Python\Python311\site-packages\dgl\backend\pytorch\tensor.py:449: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  assert input.numel() == input.storage().size(), (


In epoch 0, loss: 0.685, train_acc: 0.525, train_bal_acc: 0.517, train_recall: 0.913, train_precision: 0.519 val loss: 0.668, val acc: 0.666, bal acc: 0.497, val recall: 0.911, val precision: 0.703
In epoch 5, loss: 0.670, train_acc: 0.614, train_bal_acc: 0.614, train_recall: 0.569, train_precision: 0.634 val loss: 0.686, val acc: 0.503, bal acc: 0.469, val recall: 0.553, val precision: 0.681
In epoch 10, loss: 0.660, train_acc: 0.630, train_bal_acc: 0.629, train_recall: 0.659, train_precision: 0.631 val loss: 0.678, val acc: 0.555, bal acc: 0.482, val recall: 0.659, val precision: 0.693
In epoch 15, loss: 0.648, train_acc: 0.637, train_bal_acc: 0.637, train_recall: 0.653, train_precision: 0.642 val loss: 0.684, val acc: 0.549, bal acc: 0.481, val recall: 0.649, val precision: 0.692
In epoch 20, loss: 0.638, train_acc: 0.640, train_bal_acc: 0.640, train_recall: 0.661, train_precision: 0.642 val loss: 0.687, val acc: 0.556, bal acc: 0.480, val recall: 0.666, val precision: 0.692


KeyboardInterrupt: 